In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler
import numpy as np
import torchvision
from torchvision import datasets, models, transforms
import matplotlib.pyplot as plt
import time
import os
import copy

In [ ]:
## Splitting the data

from sklearn.model_selection import train_test_split

# Gather all image paths and their corresponding labels
image_paths = []
labels = []
class_names = ["OSCC", "with_dysplasia", "without_dysplasia"]

for class_name in class_names:
    class_path = os.path.join('/content/drive/MyDrive/Oral Disease Detection Project/', class_name)
    for img_name in os.listdir(class_path):
        image_paths.append(os.path.join(class_path, img_name))
        labels.append(class_name)

# Map string labels to numerical IDs
label_to_id = {name: i for i, name in enumerate(class_names)}
numerical_labels = [label_to_id[label] for label in labels]

# Split into training and test sets first
train_paths, test_paths, train_labels, test_labels = train_test_split(
    image_paths, numerical_labels, test_size=0.15, random_state=42, stratify=numerical_labels
)

# Then split the training set into training and validation sets
train_paths, val_paths, train_labels, val_labels = train_test_split(
    train_paths, train_labels, test_size=(0.15/0.85), random_state=42, stratify=train_labels
)

print(f"Train images: {len(train_paths)}")
print(f"Validation images: {len(val_paths)}")
print(f"Test images: {len(test_paths)}")

Train images: 2633
Validation images: 565
Test images: 565


In [ ]:
## Define Transforms

transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406],
                         std=[0.229, 0.224, 0.225])
])

In [ ]:
## Define a custom dataset
from torch.utils.data import Dataset, DataLoader
from PIL import Image

class OralDiseaseDataset(Dataset):
    def __init__(self, image_paths, labels, transform=None):
        self.image_paths = image_paths
        self.labels = labels
        self.transform = transform

    def __len__(self):
        return len(self.image_paths)

    def __getitem__(self, idx):
        img_path = self.image_paths[idx]
        label = self.labels[idx]

        image = Image.open(img_path).convert('RGB')
        if self.transform:
            image = self.transform(image)

        return image, label

In [ ]:
## Create Datasets

train_dataset = OralDiseaseDataset(train_paths, train_labels, transform=transform)
val_dataset = OralDiseaseDataset(val_paths, val_labels, transform=transform)
test_dataset = OralDiseaseDataset(test_paths, test_labels, transform=transform)

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

In [ ]:
# Load ResNet50 model
model = models.resnet50(pretrained=True)

# Modify the final classification layer
num_ftrs = model.fc.in_features
model.fc = nn.Linear(num_ftrs, 3)

for param in model.parameters():
    param.requires_grad = True

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
model.to(device)

# Define Loss Function & Optimizer
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.0001)

print("Model setup for fine-tuning complete.")
print(f"Model moved to: {device}")

/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/resnet50-0676ba61.pth" to /root/.cache/torch/hub/checkpoints/resnet50-0676ba61.pth
100%|██████████| 97.8M/97.8M [00:00<00:00, 112MB/s]


Model setup for fine-tuning complete.
Model moved to: cuda:0


In [ ]:
from tqdm import tqdm

num_epochs = 10

best_model_wts = copy.deepcopy(model.state_dict())
best_acc = 0.0

for epoch in range(num_epochs):
    print(f'Epoch {epoch}/{num_epochs - 1}')
    print('-' * 10)

    for phase in ['train', 'val']:
        if phase == 'train':
            model.train()
            dataloader = train_loader
        else:
            model.eval()
            dataloader = val_loader

        running_loss = 0.0
        running_corrects = 0

        # Iterate over data.
        for inputs, labels in tqdm(dataloader, desc=f"{phase} Phase"):
            inputs = inputs.to(device)
            labels = labels.to(device)

            # zero the parameter gradients
            optimizer.zero_grad()

            # forward
            with torch.set_grad_enabled(phase == 'train'):
                outputs = model(inputs)
                _, preds = torch.max(outputs, 1)
                loss = criterion(outputs, labels)

                # backward + optimize only if in training phase
                if phase == 'train':
                    loss.backward()
                    optimizer.step()

            # statistics
            running_loss += loss.item() * inputs.size(0)
            running_corrects += torch.sum(preds == labels.data)

        epoch_loss = running_loss / len(dataloader.dataset)
        epoch_acc = running_corrects.double() / len(dataloader.dataset)

        print(f'{phase} Loss: {epoch_loss:.4f} Acc: {epoch_acc:.4f}')

        # deep copy the model if it's the best validation accuracy
        if phase == 'val' and epoch_acc > best_acc:
            best_acc = epoch_acc
            best_model_wts = copy.deepcopy(model.state_dict())

print('Training complete')
print(f'Best val Acc: {best_acc:.4f}')

Epoch 0/9
----------


train Phase: 100%|██████████| 83/83 [08:34<00:00,  6.20s/it]


train Loss: 0.6298 Acc: 0.7159


val Phase: 100%|██████████| 18/18 [01:38<00:00,  5.49s/it]


val Loss: 0.5199 Acc: 0.7965
Epoch 1/9
----------


...

val Loss: 0.5180 Acc: 0.8743
Epoch 9/9
----------


train Phase: 100%|██████████| 83/83 [01:02<00:00,  1.32it/s]


train Loss: 0.0226 Acc: 0.9928


val Phase: 100%|██████████| 18/18 [00:09<00:00,  1.86it/s]

val Loss: 0.5269 Acc: 0.8761
Training complete
Best val Acc: 0.8903


In [ ]:
# Load best model weights
model.load_state_dict(best_model_wts)

# Evaluation on test set
model.eval()
test_running_corrects = 0
test_predictions = []
test_true_labels = []

with torch.no_grad():
    for inputs, labels in tqdm(test_loader, desc="Test Phase"):
        inputs = inputs.to(device)
        labels = labels.to(device)

        outputs = model(inputs)
        _, preds = torch.max(outputs, 1)

        test_running_corrects += torch.sum(preds == labels.data)
        test_predictions.extend(preds.cpu().numpy())
        test_true_labels.extend(labels.cpu().numpy())

test_acc = test_running_corrects.double() / len(test_loader.dataset)
print(f'Test Acc: {test_acc:.4f}')

Test Phase: 100%|██████████| 18/18 [01:52<00:00,  6.26s/it]

Test Acc: 0.8690


In [ ]:
## Define transforms to Augument Data to improve performance
train_transforms = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(15),
    transforms.ColorJitter(brightness=0.1, contrast=0.1),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

# Define transforms for validation and test data without augumentation
val_test_transforms = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

In [ ]:
# Create dataset instances
train_dataset = OralDiseaseDataset(train_paths, train_labels, train_transforms)
val_dataset = OralDiseaseDataset(val_paths, val_labels, val_test_transforms)
test_dataset = OralDiseaseDataset(test_paths, test_labels, val_test_transforms)

# Create data loaders
batch_size = 32
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

In [ ]:
num_epochs = 10

best_model_wts = copy.deepcopy(model.state_dict())
best_acc = 0.0

for epoch in range(num_epochs):
    print(f'Epoch {epoch}/{num_epochs - 1}')
    print('-' * 10)

    for phase in ['train', 'val']:
        if phase == 'train':
            model.train()
            dataloader = train_loader
        else:
            model.eval()
            dataloader = val_loader

        running_loss = 0.0
        running_corrects = 0

        # Iterate over data.
        for inputs, labels in tqdm(dataloader, desc=f"{phase} Phase"):
            inputs = inputs.to(device)
            labels = labels.to(device)

            # zero the parameter gradients
            optimizer.zero_grad()

            # forward
            with torch.set_grad_enabled(phase == 'train'):
                outputs = model(inputs)
                _, preds = torch.max(outputs, 1)
                loss = criterion(outputs, labels)

                # backward + optimize only if in training phase
                if phase == 'train':
                    loss.backward()
                    optimizer.step()

            # statistics
            running_loss += loss.item() * inputs.size(0)
            running_corrects += torch.sum(preds == labels.data)

        epoch_loss = running_loss / len(dataloader.dataset)
        epoch_acc = running_corrects.double() / len(dataloader.dataset)

        print(f'{phase} Loss: {epoch_loss:.4f} Acc: {epoch_acc:.4f}')

        # deep copy the model if it's the best validation accuracy
        if phase == 'val' and epoch_acc > best_acc:
            best_acc = epoch_acc
            best_model_wts = copy.deepcopy(model.state_dict())

print('Training complete')
print(f'Best val Acc: {best_acc:.4f}')

Epoch 0/9
----------


train Phase: 100%|██████████| 83/83 [01:07<00:00,  1.22it/s]


train Loss: 0.5368 Acc: 0.8014


val Phase: 100%|██████████| 18/18 [00:09<00:00,  1.99it/s]


val Loss: 0.4980 Acc: 0.8319
Epoch 1/9
----------


...

val Loss: 0.4889 Acc: 0.8655
Epoch 9/9
----------


train Phase: 100%|██████████| 83/83 [01:06<00:00,  1.25it/s]


train Loss: 0.1039 Acc: 0.9609


val Phase: 100%|██████████| 18/18 [00:09<00:00,  1.92it/s]

val Loss: 0.3804 Acc: 0.8885
Training complete
Best val Acc: 0.8973


In [ ]:
test_running_corrects = 0
test_predictions = []
test_true_labels = []

model.eval()

with torch.no_grad():
    for inputs, labels in tqdm(test_loader, desc="Test Phase"):
        inputs = inputs.to(device)
        labels = labels.to(device)

        outputs = model(inputs)
        _, preds = torch.max(outputs, 1)

        # ✅ Convert tensor to number using `.item()`
        test_running_corrects += torch.sum(preds == labels).item()
        test_predictions.extend(preds.cpu().numpy())
        test_true_labels.extend(labels.cpu().numpy())

test_acc = test_running_corrects / len(test_loader.dataset)
print(f'Test Accuracy: {test_acc:.4f} ({test_acc * 100:.2f}%)')

Test Phase: 100%|██████████| 18/18 [00:09<00:00,  1.96it/s]

Test Accuracy: 0.9168 (91.68%)


In [ ]:
# Further evaluation metrics (Precision, Recall, F1-score, Confusion Matrix)
from sklearn.metrics import classification_report, confusion_matrix
print("\nClassification Report on Test Set:")
print(classification_report(test_true_labels, test_predictions, target_names=class_names))

print("\nConfusion Matrix on Test Set:")
print(confusion_matrix(test_true_labels, test_predictions))


Classification Report on Test Set:
                   precision    recall  f1-score   support

             OSCC       0.95      0.92      0.93       169
   with_dysplasia       0.91      0.93      0.92       290
without_dysplasia       0.90      0.87      0.88       106

         accuracy                           0.92       565
        macro avg       0.92      0.91      0.91       565
     weighted avg       0.92      0.92      0.92       565


Confusion Matrix on Test Set:
[[155  14   0]
 [  9 271  10]
 [  0  14  92]]


In [ ]:
torch.save(model, "/content/drive/MyDrive/Oral Disease Detection Project/resnet50_oral_disease_entire_model.pth")

In [ ]:
# prompt: collect the versions of all the libaries and frameworks used in the notebook

print("Torch version:", torch.__version__)
print("Torchvision version:", torchvision.__version__)
print("Numpy version:", np.__version__)
print("PIL version:", Image.__version__)

Torch version: 2.6.0+cu124
Torchvision version: 0.21.0+cu124
Numpy version: 2.0.2
PIL version: 11.2.1
